Auteur: Michaël Leroy


Modification du traitement pour la création des edges graph communes, essai sur geodataframe pour profiter de la vectorisation:
        testé jusqu'a la recherche de paths... qui n'est pas vect et du coup hyper long (j'ai killé au bout de 3h)

    Création geodataframe Edges:

        calcul des edges attributes:
            - shortest path depuis le graph routier => distance, durée
            - geométrie: somme des LineString
            - length : somme des longeurs de chaque tronçons en mètres

[+] A faire:
        
        - prolonger les geometries des linestrings routes jusqu'au points centraux communes si on veut se servir des tracés sur une carte, sinon pas grave.

[-] Cons:
        
        40 min pour 250 nodes et 1219 edges

        -> temps estimé pour 170188 edges ~76h


In [ ]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
# AREA

In [ ]:
# data_path = './datas'
# file_name = 'decoupage_administratif'


# communes = gpd.read_feather(os.path.join(data_path,f'[{CFG.area_name}]{file_name}_osmid_centre.feather'))

# communes.crs

In [ ]:
# metro = Name2num.values()
# communes = communes.query("dep in @metro")


# # MANUAL FIX FOR MISSING OSMID ON Nominatim resquests failures
# communes.loc[5228 , 'osmid'] = 555489493
# communes.loc[5229 , 'osmid'] = 125363211
# communes.loc[10916, 'osmid'] = 9096959355
# communes.loc[34954, 'osmid'] = 41978755

# # Test on sample
# # deps = ['56']  #+['51','52','08','10']
# # communes = communes.query("dep in @deps")

# communes['osmid'] = communes['osmid'].astype(int)

# display(communes.head(1))
# display(communes.info())
# communes.shape

In [ ]:

Nodes = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.pkl'))
Edges = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes.pkl'))
# G_communes = ox.graph_from_gdfs(Nodes, Edges)


In [ ]:
Nodes.info()

In [ ]:
Edges.info()

In [ ]:
# ox.plot.plot_graph(G_communes, ax=None, figsize=(8, 8), bgcolor='#111111', node_color='w', node_size=15, node_alpha=None, node_edgecolor='none', node_zorder=1, edge_color='#999999', edge_linewidth=1, edge_alpha=None, show=True, close=False, save=False, filepath=None, dpi=300, bbox=None)


In [ ]:
from sklearn.neighbors import BallTree

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    tree = BallTree(candidates, leaf_size=20, metric='haversine')
    distances, indices = tree.query(src_points, k=k_neighbors)
    if src_points.shape[0] > 1 :
        distances = distances.transpose()
        indices = indices.transpose()
    closest = indices[0]
    closest_dist = distances[0]
    return (closest, closest_dist)

In [ ]:
Nodes_routier = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl'))
Graph_routier = ox.graph_from_gdfs(
    Nodes_routier, 
    joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'))
    )

In [ ]:
display(Nodes_routier.info(),Nodes_routier.head(1))

In [ ]:
for u, v,  d in Graph_routier.edges(data=True):
    print(u, v, d)

    break

In [ ]:
Nodes.index = Nodes.index.astype(int)

display(Nodes.info(),Nodes.head(5))

In [ ]:
import numpy as np

def get_nearest(src_points, candidates= Nodes_routier[['x', 'y']], k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    tree = BallTree(candidates, leaf_size=20, metric='haversine')
    distances, indices = tree.query(src_points, k=k_neighbors)
    if src_points.shape[0] > 1 :
        # distances = distances.transpose()
        indices = indices.transpose()
    closest = indices[0]
    # closest_dist = distances[0]
    return closest

def get_closest_routier( start,end,Nodes=Nodes,nodes_routier=Nodes_routier):
    coord = ['x', 'y']
    candidates = nodes_routier[coord]
    closest_routier, _ = get_nearest(
                        np.array([Nodes.loc[start,coord].astype(float),
                                Nodes.loc[end,coord].astype(float)
                                ]),  
                        candidates
    )    
    return nodes_routier.iloc[closest_routier[0]].name, nodes_routier.iloc[closest_routier[1]].name


start = 147124
end = 146573

source, target = get_closest_routier(start,end,Nodes, Nodes_routier)
source, target 

In [ ]:



get_nearest(Edges.xy_nodes_u)

In [ ]:
from shapely import MultiLineString, ops

# def get_path_attributes( arg, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
#     u,v = arg
#     source, target = get_closest_routier(start,end,nodes, nodes_routier,)
#     path = nx.shortest_path(graph_routier, 
#                             source=source, 
#                             target=target, 
#                             weight='travel_time', 
#                             method='dijkstra')
#     lines = ox.utils_graph.get_route_edge_attributes(
#                     graph_routier, 
#                     path, 
#                     attribute='geometry', 
#                     minimize_key='length', 
#                     retrieve_default=None)
#     multi_line = MultiLineString(lines)
#     merged_line = ops.linemerge(multi_line)

#     length = sum(
#         ox.utils_graph.get_route_edge_attributes(
#                     graph_routier, 
#                     path, 
#                     attribute='length', 
#                     minimize_key='length', 
#                     retrieve_default=None)
#     )
    
#     return merged_line, length

def get_nearest(src_points, candidates= Nodes_routier[['x', 'y']], k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    tree = BallTree(candidates, leaf_size=20, metric='haversine')
    distances, indices = tree.query(src_points, k=k_neighbors)
    if src_points.shape[0] > 1 :
        # distances = distances.transpose()
        indices = indices.transpose()
    closest = indices[0]
    # closest_dist = distances[0]
    return closest


def get_closest_routier( u ,Nodes=Nodes,nodes_routier=Nodes_routier):
    coord = ['x', 'y']
    candidates = nodes_routier[coord]
    closest_routier, _ = get_nearest( 
                            np.array([Nodes.loc[u,coord].astype(float).values]).reshape(-1,1),

                        # np.array([Nodes.loc[start,coord].astype(float),
                                # Nodes.loc[end,coord].astype(float)
                                # ]),  
                        candidates
    )    
    return nodes_routier.iloc[closest_routier[0]].name


def get_source_target(arg, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier):
    u,v = arg
    return get_closest_routier(start,end,nodes, nodes_routier,)


def get_path( arg, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
    source, target = arg
    # source, target = get_closest_routier(start,end,nodes, nodes_routier,)
    path = nx.shortest_path(graph_routier, 
                            source=source, 
                            target=target, 
                            weight='travel_time', 
                            method='dijkstra')
    return path
    

def get_path_line( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):


    lines = ox.utils_graph.get_route_edge_attributes(
                    graph_routier, 
                    path, 
                    attribute='geometry', 
                    minimize_key='length', 
                    retrieve_default=None)
    multi_line = MultiLineString(lines)
    merged_line = ops.linemerge(multi_line)
    return merged_line


def get_path_length( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):

    length = sum(
        ox.utils_graph.get_route_edge_attributes(
                    graph_routier, 
                    path, 
                    attribute='length', 
                    minimize_key='length', 
                    retrieve_default=None)
    )
    
    return length


In [ ]:
backup = Edges.copy()


In [ ]:
Edges = backup.copy()

In [ ]:
# Edges = Edges.sample(5)
# Edges

In [ ]:
Edges.reset_index(inplace=True)
Edges.sample(2)

In [ ]:
Edges['x_nodes_u'] = Edges.u.apply(lambda x: Nodes.loc[int(x),'x'])
Edges['y_nodes_u'] = Edges.u.apply(lambda x: Nodes.loc[int(x),'y'])

Edges['x_nodes_v'] = Edges.v.apply(lambda x: Nodes.loc[int(x),'x'])
Edges['y_nodes_v'] = Edges.v.apply(lambda x: Nodes.loc[int(x),'y'])

Edges.sample(2)

In [ ]:
tree = BallTree(Nodes_routier[['x','y']], leaf_size=20, metric='haversine')


In [ ]:
Edges['src_idx'] = tree.query(Edges[['x_nodes_u','y_nodes_u']], k=1)[1]
Edges['src'] = Nodes_routier.reset_index().iloc[Edges['src_idx'].values]['osmid'].values

In [ ]:
Edges['trgt_idx'] = tree.query(Edges[['x_nodes_v','y_nodes_v']], k=1)[1]
Edges['trgt'] = Nodes_routier.reset_index().iloc[Edges['trgt_idx'].values]['osmid'].values

In [ ]:
Edges['src_trgt'] = tuple(zip(Edges.src, Edges.trgt))
Edges.sample(2)

In [ ]:
# TODO: 1 voir pour vectoriser cette étape
#            - sub classer la classe source 
#            _ modiifer le code source

Edges['path'] = Edges.src_trgt.apply(get_path)
Edges.sample(2)

In [ ]:
Edges['length'] = Edges.path.apply(get_path_length)
Edges.sample(2)

In [ ]:
Edges['line'] = Edges.path.apply(get_path_line)
Edges.sample(2)

In [ ]:
# TODO:  supprimer les colonnes intermediaires avant la sauvegarde

In [ ]:
joblib.dump(
            Edges,
            os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes.pkl')
        )

--------------------------------